### Turing.ml example: https://turing.ml/dev/tutorials/10-bayesian-differential-equations/ 

In [ ]:
using Turing, Distributions, DifferentialEquations

# Import MCMCChain, Plots, and StatsPlots for visualizations and diagnostics.
using MCMCChains, Plots, StatsPlots

# Set a seed for reproducibility.
using Random
Random.seed!(14);

In [ ]:
function lotka_volterra(du,u,p,t)
    x, y = u
    α, β, γ, δ  = p
    du[1] = (α - β*y)x # dx =
    du[2] = (δ*x - γ)y # dy =
end
p = [1.5, 1.0, 3.0, 1.0]
u0 = [1.0,1.0]
prob1 = ODEProblem(lotka_volterra,u0,(0.0,10.0),p)
sol = solve(prob1,Tsit5())
plot(sol)

sol1 = solve(prob1,Tsit5(),saveat=0.1)
odedata = Array(sol1) + 0.8 * randn(size(Array(sol1)))
plot(sol1, alpha = 0.3, legend = false); scatter!(sol1.t, odedata')

In [ ]:
Turing.setadbackend(:forwarddiff)

@model function fitlv(data, prob1)
    σ ~ InverseGamma(2, 3) # ~ is the tilde character
    α ~ truncated(Normal(1.5,0.5),0.5,2.5)
    β ~ truncated(Normal(1.2,0.5),0,2)
    γ ~ truncated(Normal(3.0,0.5),1,4)
    δ ~ truncated(Normal(1.0,0.5),0,2)

    p = [α,β,γ,δ]
    prob = remake(prob1, p=p)
    predicted = solve(prob,Tsit5(),saveat=0.1)

    for i = 1:length(predicted)
        data[:,i] ~ MvNormal(predicted[i], σ)
    end
end

model = fitlv(odedata, prob1)

# This next command runs 3 independent chains without using multithreading.
chain = mapreduce(c -> sample(model, NUTS(.65),1000), chainscat, 1:3)
plot(chain)

In [ ]:
pl = scatter(sol1.t, odedata');
chain_array = Array(chain)
for k in 1:300
    resol = solve(remake(prob1,p=chain_array[rand(1:1500), 1:4]),Tsit5(),saveat=0.1)
    plot!(resol, alpha=0.1, color = "#BBBBBB", legend = false)
end
# display(pl)
plot!(sol1, w=1, legend = false)

### simple reaction ODE


In [ ]:
using Turing, Distributions, DifferentialEquations

# Import MCMCChain, Plots, and StatsPlots for visualizations and diagnostics.
using MCMCChains, Plots, StatsPlots

# Set a seed for reproducibility.
using Random
Random.seed!(14);

function rxn(du, u, p, t)
    v = 0.0001
    OF = u[1]
    H_out = u[2]
    OFHb = u[3]
    k_on = 10^p[1]
    k_off = 10^p[2]
    du[1] = v*((OFHb*k_off) - (OF*H_out*k_on))
    du[2] = v*((OFHb*k_off) - (OF*H_out*k_on))
    du[3] = v*((OF*H_out*k_on) - (OFHb*k_off)) 
end


p = [6,2]  # log10 rates
u0 = [1.0,1.0, 0.0]
tspan = (0.0,1.0)
prob1 = ODEProblem(rxn,u0,tspan,p)
condition(u,t,integrator) = t==0.5
affect!(integrator) = integrator.u[1] = u0[1]
cb1 = DiscreteCallback(condition,affect!)

sol1 = solve(prob1,AutoTsit5(Rosenbrock23()),callback=cb1,tstops=[0.5])
odedata = Array(sol1) + 0.01 * randn(size(Array(sol1)))
# plot(sol1, alpha = 0.3, legend = false); scatter!(sol1.t, odedata')

plot(sol1, alpha = 0.8, legend = true)


#### Sampling and analysis

In [ ]:
Turing.setadbackend(:forwarddiff)

@model function fitlv(data, prob1)
    σ ~ Uniform(0.05, 0.5) # ~ is the tilde character
    log_k_on ~ Uniform(5,8)
    log_k_off ~ Uniform(1,4)

    p = [σ,log_k_on,log_k_off]
    prob = remake(prob1, p=p)
    condition(u,t,integrator) = t==0.5
    affect!(integrator) = integrator.u[1] = 1
    cb = DiscreteCallback(condition,affect!)

    predicted = solve(prob1,AutoTsit5(Rosenbrock23()),callback=cb1,tstops=[0.5])

    for i = 1:length(predicted)
        data[:,i] ~ MvNormal(predicted[i], σ)
    end
end

model = fitlv(odedata, prob1)

# This next command runs 3 independent chains without using multithreading.
chain = mapreduce(c -> sample(model, NUTS(.65),10000), chainscat, 1:3)
plot(chain)

In [ ]:
pl = scatter(sol1.t, odedata');

chain_array = Array(chain)
for k in 1:10
    prob = remake(prob1, p=chain_array[rand(1:1500), 1:2])
    condition(u,t,integrator) = t==0.5
    affect!(integrator) = integrator.u[1] = 1
    cb = DiscreteCallback(condition,affect!)
    predicted2 = solve(prob1,AutoTsit5(Rosenbrock23()),callback=cb1,tstops=[0.5])
    plot!(predicted2, alpha=0.1, color = "#BBBBBB", legend = false)
end
# display(pl)
plot!(sol1, w=1, legend = false)

### Single cycle transporter ODEs

In [ ]:
using Turing, Distributions, DifferentialEquations, Sundials

# Import MCMCChain, Plots, and StatsPlots for visualizations and diagnostics.
using MCMCChains, Plots, StatsPlots

# Set a seed for reproducibility.
using Random
Random.seed!(14);

function transporter_s1(du, u, p, t)
    ### compartment size (in L)
    vol = 0.0001
    v=1

    ### model parameters - reaction rate constants (in 1/s and 1/(Ms))
    rxn1_k1 = 0
    rxn1_k2 = 0
    rxn2_k1 = p[1]  # H on rate
    rxn2_k2 = p[2]  # H off rate
    rxn3_k1 = p[3]  # S off rate
    rxn3_k2 = p[4]  # S on rate
    rxn4_k1 = p[5]  # conf rate
    rxn4_k2 = p[6]  # conf rate
    rxn5_k1 = 0
    rxn5_k2 = 0
    rxn6_k1 = p[7]  # conf rate
    rxn6_k2 = p[8]  # conf rate
    rxn7_k1 = 0
    rxn7_k2 = 0
    rxn8_k1 = 0
    rxn8_k2 = 0
    rxn9_k1 = 0
    rxn9_k2 = 0
    rxn10_k1 = 0
    rxn10_k2 = 0
    rxn11_k1 = p[9]  # S on rate
    rxn11_k2 = p[10]  # S off rate
    rxn12_k1 = p[11]  # H off rate
    rxn12_k2 = (rxn2_k1*rxn3_k1*rxn4_k1*rxn6_k1*rxn11_k1*rxn12_k1)/(rxn2_k2*rxn3_k2*rxn4_k2*rxn6_k2*rxn11_k2)  # H on rate (cycle constraint)
  
    ### model state concentrations (in M)
    H_out = 1e-7  # constant external ion concentration (e.g. buffer solution) 
    S_out = 0.001  # constant external substrate concentration (e.g. buffer solution) 
    
    H_in = u[1]  # internal ion concentration
    S_in = u[2]  # internal substrate concentation
    OF = u[3]  # fully unbound outward-facing transporter 
    IF = u[4]  # fully unbound inward-facing transporter
    OF_Hb = u[5]  # ion-bound only outward-facing transporter
    IF_Hb = u[6]  # ion-bound only inward-facing transporter
    OF_Sb = u[7]  # substrate-bound only outward-facing transporter
    IF_Sb = u[8]  # substrate-bound only inward-facing transporter
    OF_Hb_Sb = u[9]  # ion and substrate bound outward-facing transporter
    IF_Hb_Sb = u[10]  # ion and substrate bound inward-facing transporter

           
    ### reaction equations (from Tellurium)       
    vrxn1 = vol*(rxn1_k1*IF-rxn1_k2*OF)
    vrxn2 = vol*(rxn2_k1*OF*H_out-rxn2_k2*OF_Hb)
    vrxn3 = vol*(rxn3_k1*OF_Sb-rxn3_k2*OF*S_out)
    vrxn4 = vol*(rxn4_k1*OF_Hb-rxn4_k2*IF_Hb)
    vrxn5 = vol*(rxn5_k1*OF_Hb_Sb-rxn5_k2*OF_Hb*S_out)
    vrxn6 = vol*(rxn6_k1*IF_Sb-rxn6_k2*OF_Sb)
    vrxn7 = vol*(rxn7_k1*OF_Sb*H_out-rxn7_k2*OF_Hb_Sb)
    vrxn8 = vol*(rxn8_k1*OF_Hb_Sb-rxn8_k2*IF_Hb_Sb)
    vrxn9 = vol*(rxn9_k1*IF_Hb-rxn9_k2*IF*H_in)
    vrxn10 = vol*(rxn10_k1*IF*S_in-rxn10_k2*IF_Sb)
    vrxn11 = vol*(rxn11_k1*IF_Hb*S_in-rxn11_k2*IF_Hb_Sb)
    vrxn12 = vol*(rxn12_k1*IF_Hb_Sb-rxn12_k2*IF_Sb*H_in)

    ### ODE equations (from Tellurium)
    du[1] = v*(vrxn1 - vrxn2 + vrxn3)
    du[2] = v*(vrxn2 - vrxn4 + vrxn5)
    du[3] = v*(vrxn4 - vrxn9 - vrxn11)
    du[4] = v*(-vrxn10 - vrxn11)
    du[5] = v*(vrxn8 + vrxn11 - vrxn12)
    du[6] = v*(vrxn9 + vrxn12)
    du[7] = v*(-vrxn6 + vrxn10 + vrxn12)
    du[8] = v*(-vrxn3 + vrxn6 - vrxn7)
    du[9] = v*(-vrxn1 + vrxn9 - vrxn10)
    du[10] = v*(-vrxn5 + vrxn7 - vrxn8)
end 


### rate constants
k_H_on = 1e10
k_H_off = 1e3
k_S_on = 1e7
k_S_off = 1e3
k_conf = 1e2
p = [
    k_H_on,
    k_H_off,
    k_S_off,
    k_S_on,
    k_conf,
    k_conf,
    k_conf,
    k_conf,
    k_S_on,  
    k_S_off,  
    k_H_off,  
]
std_true = 1e-13 

### initial parameters
u0 = [
    1e-7,
    1e-3,
    2.833e-08,
    2.125e-08,
    2.833e-08,
    2.833e-08,
    2.125e-08,
    2.125e-08,
    2.125e-08,
    2.833e-08,
]

tspan = (0.0,5.0)
prob = ODEProblem(transporter_s1,u0,tspan,p)
sol = solve(prob, CVODE_BDF(), saveat=0.01, abstol = 1e-16, reltol = 1e-14, progress=true, )
